In [1]:
%run ../utilities/web_utilities.ipynb
%run ../utilities/database_utilities.ipynb
%run ../utilities/encoder_utilities.ipynb
%run ../utilities/pd_utilities.ipynb

In [2]:
write_mode = 'replace'
target_table = 'whl_player_game_stats'

In [3]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../../')

import settings as st

In [4]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
db_host = st.database['local']['host']
db_port = st.database['local']['port']
db_user = st.database['local']['user']
db_pass = st.database['local']['pass']
db_sys = st.database['local']['system']
db_db = st.database['local']['db']

In [6]:
key = "41b145a848f4bd67"

In [7]:
sql = "SELECT DISTINCT player_id FROM whl_team_roster_by_season WHERE player_id IS NOT NULL ORDER BY player_id ASC"
res = query_database_to_list(db_sys, db_user, db_pass, db_host, db_port, db_db, sql)

In [8]:
master_df = pd.DataFrame()

In [9]:
i = 1
for r in res:
    player_id = str(r['player_id']).replace(".0", "")
    
    print(i, player_id)
    
    player_id = "27537"
    
    url = "http://lscluster.hockeytech.com/feed/?feed=modulekit&view=player&key={0}&fmt=json&client_code=whl&lang=en&player_id={1}&category=gamebygame".format(key, player_id)

    json_data = return_json(url)
    
    for season in json_data['SiteKit']['Player']['seasons_played']:
        season_id = season['season_id']
        
        print(season_id)
        
        season_url = 'http://lscluster.hockeytech.com/feed/?feed=modulekit&view=player&key={0}&fmt=json&client_code=whl&lang=en&player_id={1}&category=gamebygame&season_id={2}'.format(key, player_id, season_id)
    
        season_data = return_json(season_url)

        df = pd.DataFrame(season_data['SiteKit']['Player']['games'])
        
        df['player_id'] = player_id
        df['season_id'] = season_id
        
        master_df = master_df.append(df)

    break

1 21812
267
266
265
264
262
261
257
256


In [10]:
"""
Clean up Data
"""
master_df['date_played'] = master_df.date_played.apply(lambda x: clearDate(x))

"""
Fix Data Types
"""
master_df["assists"] = pd.to_numeric(master_df["assists"])
master_df["empty_net_goals"] = pd.to_numeric(master_df["empty_net_goals"])
master_df["faceoffs_taken"] = pd.to_numeric(master_df["faceoffs_taken"])
master_df["faceoffs_won"] = pd.to_numeric(master_df["faceoffs_won"])
master_df["first_goals_scored"] = pd.to_numeric(master_df["first_goals_scored"])
master_df["game_tieing_goals"] = pd.to_numeric(master_df["game_tieing_goals"])
master_df["game_winning_goals"] = pd.to_numeric(master_df["game_winning_goals"])
master_df["goalie"] = pd.to_numeric(master_df["goalie"])
master_df["goals"] = pd.to_numeric(master_df["goals"])
master_df["home"] = pd.to_numeric(master_df["home"])
master_df["home_team"] = pd.to_numeric(master_df["home_team"])
master_df["id"] = pd.to_numeric(master_df["id"])
master_df["insurange_goals"] = pd.to_numeric(master_df["insurange_goals"])
master_df["penalty_minutes"] = pd.to_numeric(master_df["penalty_minutes"])
master_df["player_id"] = pd.to_numeric(master_df["player_id"])
master_df["player_team"] = pd.to_numeric(master_df["player_team"])
master_df["plus_minus"] = pd.to_numeric(master_df["plus_minus"])
master_df["plusminus"] = pd.to_numeric(master_df["plusminus"])
master_df["points"] = pd.to_numeric(master_df["points"])
master_df["power_play_goals"] = pd.to_numeric(master_df["power_play_goals"])
master_df["season_id"] = pd.to_numeric(master_df["season_id"])
master_df["shooting_percentage"] = pd.to_numeric(master_df["shooting_percentage"])
master_df["shootout_attempts"] = pd.to_numeric(master_df["shootout_attempts"])
master_df["shootout_goals"] = pd.to_numeric(master_df["shootout_goals"])
master_df["shootout_goals_win"] = pd.to_numeric(master_df["shootout_goals_win"])
master_df["shootout_shots"] = pd.to_numeric(master_df["shootout_shots"])
master_df["shootout_shots_percentage"] = pd.to_numeric(master_df["shootout_shots_percentage"])
master_df["short_handed_goals"] = pd.to_numeric(master_df["short_handed_goals"])
master_df["shots"] = pd.to_numeric(master_df["shots"])
master_df["visiting_team"] = pd.to_numeric(master_df["visiting_team"])

master_df["date_played"] = pd.to_datetime(master_df["date_played"])

In [11]:
master_df = master_df.reset_index(drop=True)

In [12]:
if write_mode == 'append':
    index = get_table_new_id(db_sys, db_user, db_pass, db_host, db_port, db_db, target_table, 'index')
    master_df.insert(0, 'index', master_df.index + index)
else:
    master_df.insert(0, 'index', master_df.index)

In [13]:
write_df_to_database(master_df, target_table, db_sys, db_user, db_pass, db_host, db_port, db_db, write_mode, False)